### Q2.1

In [ ]:
import glob
import cv2
import numpy as np
from scipy import misc

files = glob.glob("./cifar10_transformed/masks/*.png")

data = []
file_data = glob.glob("./cifar10_transformed/imgs/*.png")
for i in range(len(file_data)):
    name = file_data[i]
    data.append(cv2.imread(name))

data = np.asarray(data)
print(data.shape)


masks = []
masks_reshaped = []
for i in range(len(files)):
    temp_mask = misc.imread(files[i])
    masks_reshaped.append(np.ravel(temp_mask))
    
masks_reshaped = np.asarray(masks_reshaped)
print(masks_reshaped.shape)

## 0s and 1s masks

indices = []
threshold_masks = []
two = tf.constant(2,tf.float64)
for i in range(len(masks_reshaped)):
    ind = np.asarray(np.where(masks_reshaped[i] < 2))
    mask_tensor = masks_reshaped[i][ind].astype(float)
    threshold_masks.append(mask_tensor.reshape(-1))
    ind = ind.reshape(-1)
    indices.append(ind)

In [ ]:
padded_masks = []
for i in range(len(threshold_masks)):
    temp = np.zeros(36)
    temp[:len(threshold_masks[i])] = threshold_masks[i]
    temp[len(threshold_masks[i]):] = np.zeros(36 - len(threshold_masks[i]))
    padded_masks.append(temp)

print(len(padded_masks))

In [1]:
import tensorflow as tf


def batch_norm(x, is_train=True, decay=0.99, epsilon=0.001):
    shape_x = x.get_shape().as_list()
    beta = tf.get_variable('beta', shape_x[-1], initializer=tf.constant_initializer(0.0))
    gamma = tf.get_variable('gamma', shape_x[-1], initializer=tf.constant_initializer(1.0))
    moving_mean = tf.get_variable('moving_mean', shape_x[-1],
                initializer=tf.constant_initializer(0.0), trainable=False)
    moving_var = tf.get_variable('moving_var', shape_x[-1],
               initializer=tf.constant_initializer(1.0), trainable=False)

    if is_train:
        mean, var = tf.nn.moments(x, np.arange(len(shape_x)-1), keep_dims=True)
        mean = tf.reshape(mean, [mean.shape.as_list()[-1]])
        #mean = tf.reshape(mean, [mean.get_shape().as_list()[-1]])
        var = tf.reshape(var, [var.shape.as_list()[-1]])

        update_moving_mean = tf.assign(moving_mean, moving_mean*decay + mean*(1-decay))
        update_moving_var = tf.assign(moving_var,
                            moving_var*decay + shape_x[0]/(shape_x[0]-1)*var*(1-decay))
        update_ops = [update_moving_mean, update_moving_var]

        with tf.control_dependencies(update_ops):
            return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

    else:
        mean = moving_mean
        var = moving_var
        return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

In [68]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm

slim = tf.contrib.slim
mu = 0
sigma = 0.1
#weights = tf.Variable(tf.truncated_normal(shape=(32, 43), mean = mu, stddev = sigma))
img_index = 0
pred_global = 0
def cifarNet(x, y, is_train=True):
    two = tf.constant(2,tf.float32)
    vs = tf.get_variable_scope()
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='SAME') + conv1_b
    conv1 = batch_norm(conv1)
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 2
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 32, 64), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(64))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='SAME') + conv2_b
    conv2 = batch_norm(conv2)
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 3
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 64, 128), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(128))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='SAME') + conv3_b
    
    conv3 = batch_norm(conv3)
    conv3 = tf.nn.relu(conv3)
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 4
    
    conv4_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 128, 256), mean = mu, stddev = sigma))
    conv4_b = tf.Variable(tf.zeros(256))
    
    conv4   = tf.nn.conv2d(conv3, conv4_W, strides=[1, 1, 1, 1], padding='SAME') + conv4_b
    conv4 = batch_norm(conv4)
    
    conv4 = tf.nn.relu(conv4)
    
    #Layer 5
    conv5_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 256, 256), mean = mu, stddev = sigma))
    conv5_b = tf.Variable(tf.zeros(256))
    
    conv5   = tf.nn.conv2d(conv4, conv5_W, strides=[1, 1, 1, 1], padding='SAME') + conv5_b
    conv5 = batch_norm(conv5)
    conv5 = tf.nn.relu(conv5)
    #Layer 6
    conv6_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 256, 1), mean = mu, stddev = sigma))
    conv6_b = tf.Variable(tf.zeros(1))
    
    conv6   = tf.nn.conv2d(conv5, conv6_W,strides=[1, 1, 1, 1], padding='SAME') + conv6_b

    conv6 =  flatten(conv6)
    
    
    bool_indices = tf.not_equal(y, two)
    threshold_indices = tf.where(bool_indices)
    ground_truth = tf.gather_nd(y, threshold_indices)
    predicted = tf.gather_nd(conv6,threshold_indices)
    
    return conv6, ground_truth, predicted
    

In [83]:
from sklearn.utils import shuffle

BATCH_SIZE = 100
EPOCHS = 5
rate_initial = 0.001
is_train = True
batch = tf.Variable(0)
x = tf.placeholder(tf.float32, shape=[None, 48,48,3])
y = tf.placeholder(tf.float32, shape=[None,36])
z = tf.placeholder(tf.int32, shape=[None,36])
zz = [100,200]
off = 0
ending = 100
logits, ground_truth, predicted  = cifarNet(x, y, is_train)

predicted_temp = tf.nn.sigmoid(predicted)
predicted_threshold = tf.greater(predicted_temp, 0.5)
predicted_threshold = tf.cast(predicted_threshold, tf.int32)
ground_truth_acc = tf.nn.sigmoid(ground_truth)
predicted_threshold = tf.cast(predicted_threshold, tf.float32)

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=ground_truth, logits=predicted)

loss_operation = tf.reduce_mean(cross_entropy)

rate = tf.train.exponential_decay(rate_initial, batch*BATCH_SIZE,
                                           20, 0.05, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(predicted_threshold, ground_truth)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.int64))


saver = tf.train.Saver()


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = 20000
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        accuracy = 0
        X_train, y_train,z_train = (data, masks_reshaped, padded_masks)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y, batch_z = X_train[offset:end], y_train[offset:end],z_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,z: batch_z})
            accuracy = accuracy + sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,z: batch_z})
        print("EPOCH {} ...".format(i+1))
        accuracy = accuracy / 200
        print("Training Accuracy = {:.3f}".format(accuracy))
        print()
        
    saver.save(sess, './q21Model')
    print("Model saved")

threshold:  Tensor("Where_42:0", shape=(?, 2), dtype=int64)
Training...

EPOCH 1 ...
Training Accuracy = 0.750



KeyboardInterrupt: 